In [ ]:
import pandas as pd
from dateutil import parser
result_df = pd.read_csv("../../data/clean_data.100k.csv")

## 5. Добавление новой ноды + новой связи

- Подготовить фичи для новой ноды.
- Добавить новые связи.
- Сделать inference на уже обученной GraphSAGE.

1. `new_node_features` — это вектор, который должен включать:

- числовые агрегаты ноды (total_visits, avg_time, bounce_rate и т.д.)
- категориальные коды (category)
- TF-IDF embedding (screen + feature + action + category)
- все в том же порядке, что и graph.x

2. `edge_attr` — массив признаков для новой связи, размерность = graph.edge_attr.shape[1]. Если нет особых признаков — можно заполнить нулями.

3. 'NEW' — специальный маркер для новой ноды при указании src или dst.

4. **Предсказания:**
- `new_node_pred` — прогноз churn_rate для новой ноды
- `new_edge_preds` — прогноз потока пользователей по новым ребрам

In [ ]:
# add_node_edge_inference.py
import torch
import numpy as np
from torch_geometric.data import Data

def add_new_node_and_edges(graph: Data,
                           model,
                           new_node_features: np.ndarray,
                           new_edges: list,
                           device='cpu'):
    """
    Добавляет новую ноду и новые связи в существующий граф и делает предсказания.

    Параметры:
    - graph: PyG Data объект, уже обученный
    - model: обученный GraphSAGE
    - new_node_features: 1D numpy array, длина = graph.x.shape[1]
    - new_edges: list of tuples (src_idx, dst_idx, edge_attr_array)
        - если новый узел участвует, можно использовать dst_idx='NEW' или src_idx='NEW'
    - device: 'cpu' или 'cuda'

    Возвращает:
    - new_node_pred: float, предсказанный churn_rate для новой ноды
    - new_edge_preds: list of float, предсказанные переходы по новым связям
    """

    model.eval()
    device = torch.device(device)
    graph = graph.to(device)
    model = model.to(device)

    # 1. Добавляем новую ноду
    x_new = torch.cat([graph.x.cpu(), torch.tensor(new_node_features, dtype=torch.float).unsqueeze(0)], dim=0)

    # 2. Добавляем новые связи
    edge_idx_list = [graph.edge_index.cpu().numpy()[0].tolist(),
                     graph.edge_index.cpu().numpy()[1].tolist()]
    edge_attr_list = graph.edge_attr.cpu().numpy().tolist() if graph.edge_attr is not None else []

    N = graph.num_nodes  # индекс новой ноды

    new_edge_positions = []
    for src, dst, edge_attr in new_edges:
        src_idx = N if src == 'NEW' else src
        dst_idx = N if dst == 'NEW' else dst
        edge_idx_list[0].append(src_idx)
        edge_idx_list[1].append(dst_idx)
        edge_attr_list.append(np.array(edge_attr).astype(float))
        new_edge_positions.append(len(edge_idx_list[0]) - 1)  # позиции новых ребер

    edge_index_new = torch.tensor(edge_idx_list, dtype=torch.long)
    edge_attr_new = torch.tensor(np.vstack(edge_attr_list), dtype=torch.float)

    data_new = Data(x=x_new, edge_index=edge_index_new, edge_attr=edge_attr_new).to(device)

    # 3. Forward pass
    with torch.no_grad():
        node_pred, edge_pred, _ = model(data_new.x, data_new.edge_index, data_new.edge_attr)

    # 4. Результаты
    new_node_pred = node_pred[N].cpu().item()
    new_edge_preds = [edge_pred[i].cpu().item() for i in new_edge_positions]

    return new_node_pred, new_edge_preds


In [ ]:
# Допустим, у нас уже есть обученная модель `trained_model` и Data `graph`

# 1) Подготовка признаков новой ноды
# - TF-IDF embedding: text_embedding для новой ноды
# - numeric features: среднее соседних нод или глобальное среднее
new_node_features = graph.x.mean(dim=0).cpu().numpy()  # простой пример

# 2) Подготовка новых ребер
# - edge_attr должен совпадать по размерности с graph.edge_attr
E_attr_dim = graph.edge_attr.size(1) if graph.edge_attr is not None else 0
sample_edge_attr = np.zeros(E_attr_dim, dtype=float)

# Допустим, хотим добавить связи: существующая нода 0 -> новая, новая -> нода 1
new_edges = [
    (0, 'NEW', sample_edge_attr),    # edge from node 0 -> NEW
    ('NEW', 1, sample_edge_attr)     # edge NEW -> node 1
]

# 3) Получаем предсказания
new_node_pred, new_edge_preds = add_new_node_and_edges(graph, trained_model,
                                                      new_node_features,
                                                      new_edges,
                                                      device='cpu')

print("Predicted churn_rate for new node:", new_node_pred)
print("Predicted flows for new edges:", new_edge_preds)


Predicted churn_rate for new node: 0.13257402181625366
Predicted flows for new edges: [-0.07048645615577698, -0.08026887476444244]
